In [27]:
#Downloading pretrained model
import kagglehub

# Download latest version
path = kagglehub.model_download("keras/efficientnetv2/keras/efficientnetv2_s_imagenet_classifier")

print("Path to model files:", path)

Path to model files: /Users/badalkr.sharma/.cache/kagglehub/models/keras/efficientnetv2/keras/efficientnetv2_s_imagenet_classifier/2


In [28]:
# downloading dataseet LFW
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tf_keras.utils import to_categorical

# Download the full LFW dataset with all people included (no filtering)
lfw_people=fetch_lfw_people(min_faces_per_person=0,resize=0.4,color=False)

# Get the face images and labels
data=lfw_people.images # face images
labels=lfw_people.target # people
target_names= lfw_people.target_names #list of all person names


In [29]:
#Print dataset details
print(f"Total images: {data.shape[0]}")
print(f"Image size : {data.shape[1]}x{data.shape[2]}") # heightxwidth
print(f"Number of classes (people) : {len(target_names)}")

Total images: 13233
Image size : 50x37
Number of classes (people) : 5749


In [30]:
# #Creating input data
# import tensorflow as tf
# import tf_keras as keras
# from tf_keras.preprocessing.image import ImageDataGenerator

# Image_shape=(224,224)

In [31]:
#Reshape the data to include the channel dimension (for grayscale images, channel 1)
data = data.reshape((data.shape[0], data.shape[1],data.shape[2],1))
#split tha data
x_train,x_test, y_train,y_test= train_test_split(data,labels,test_size=0.2,random_state=42)

print(f"Training data shape : {x_train.shape}")
print(f"Testing data shape: {x_test.shape}")

Training data shape : (10586, 50, 37, 1)
Testing data shape: (2647, 50, 37, 1)


In [32]:
#Create  TensorBoard callback(functionized beacuse we need to create a new onw for each model)
import datetime
from tf_keras.callbacks import TensorBoard

def create_tensorboard_callback(dir_name,experiment_name):
    log_dir= dir_name +'/'+experiment_name+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensor_callback=TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to:{log_dir}")
    return tensor_callback


### Create model 

In [33]:
import tensorflow as tf
import tf_keras as keras
import tensorflow_hub as hub
from tf_keras import layers

In [34]:
def create_model(model_url, num_classes=5749):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=Image_shape+(3,)) # define the input image shape
  
  # Create our own model
  model = keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer      
  ])

  return model

In [35]:
# # creating model 
# efficient_pre= create_model(path,
#                             num_classes=len(target_names)

### setting up callbacks

In [36]:
#Create  TensorBoard callback(functionized beacuse we need to create a new onw for each model)
import datetime
from tf_keras.callbacks import TensorBoard

def create_tensorboard_callback(dir_name,experiment_name):
    log_dir= dir_name +'/'+experiment_name+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensor_callback=TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to:{log_dir}")
    return tensor_callback


In [37]:
from tf_keras import layers, Model
import tensorflow as tf

# Create the model function
def create_model(num_classes):
    base_model = tf.keras.applications.EfficientNetV2S(
        include_top=False,
        weights='imagenet',  # Use ImageNet weights
        input_shape=(50, 37, 3)  # Make sure to set the correct shape (3 for RGB)
    )
    # Freeze the base model
    base_model.trainable = False
    
    # Build the top layers for your specific task
    inputs = layers.Input(shape=(50, 37, 3))
    x = base_model(inputs, training=False)  # Set training=False to avoid batch normalization updates
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model, base_model  # Return both the model and the base model

# Assuming you've already loaded your dataset and defined target_names
num_classes = len(target_names)  # Use the number of unique classes in your dataset

# Create the model and get the base model
model, base_model = create_model(num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the architecture
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 50, 37, 3)]          0         []                            
                                                                                                  
 tf.cast_183 (TFOpLambda)    (None, 50, 37, 3)            0         ['input_4[0][0]']             
                                                                                                  
 tf.math.multiply_423 (TFOp  (None, 50, 37, 3)            0         ['tf.cast_183[0][0]']         
 Lambda)                                                                                          
                                                                                                  
 tf.__operators__.add_513 (  (None, 50, 37, 3)            0         ['tf.math.multiply_423[0

In [43]:
import numpy as np
# Convert grayscale (1 channel) images to RGB (3 channels)
x_train_rgb = np.repeat(x_train, 3, axis=-1)
x_test_rgb = np.repeat(x_test, 3, axis=-1)

# Now x_train_rgb and x_test_rgb have the shape (samples, height, width, 3)
print(f"Training data shape after conversion: {x_train_rgb.shape}")
print(f"Testing data shape after conversion: {x_test_rgb.shape}")

Training data shape after conversion: (10586, 50, 37, 3)
Testing data shape after conversion: (2647, 50, 37, 3)


In [50]:
# Dummy batch to initialize the model
dummy_batch = np.zeros((1, 50, 37, 3))  # Adjust shape as per input size
model(dummy_batch)

<tf.Tensor: shape=(1, 5749), dtype=float32, numpy=
array([[0.00016967, 0.00016455, 0.00015667, ..., 0.00017543, 0.00017758,
        0.00017135]], dtype=float32)>

In [47]:
# Continue training with fine-tuning
history_finetune = model.fit(
    x_train_rgb,  # Use the RGB data
    y_train,
    steps_per_epoch=len(x_train) // 32,
    epochs=5,  # Fine-tune for a few more epochs
    validation_data=(x_test_rgb, y_test),  # Use the RGB data for validation
    callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                           experiment_name='efficientNet_pre')]
)

Saving TensorBoard log files to:tensorflow_hub/efficientNet_pre/20241021-142934


Epoch 1/5


AttributeError: 'Function' object has no attribute '_concrete_variable_creation_fn'

In [49]:
# Unfreeze the base model before fine-tuning
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy batch to initialize the model
dummy_batch = np.zeros((1, 50, 37, 3))  # Adjust shape as per input size
model(dummy_batch)

# Continue training with fine-tuning
history_finetune = model.fit(
    x_train_rgb,  # Use the RGB data
    y_train,
    steps_per_epoch=len(x_train) // 32,
    epochs=5,  # Fine-tune for a few more epochs
    validation_data=(x_test_rgb, y_test),  # Use the RGB data for validation
    callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                           experiment_name='efficientNet_pre')]
)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x31b9fb310>